# Настройка среды локально

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('C://work/dev/python/progs/texts/sec_bert/')

# mitre attack

In [3]:
import json
import pandas as pd
# d=json.load(open('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/enterprise-attack.json'))
d=json.load(open('data/external/enterprise-attack.json'))

d.keys()

dict_keys(['type', 'id', 'spec_version', 'objects'])

## Изучение

In [ ]:
d['objects'][0].keys()

### типы объектов

In [34]:
set([it['type'] for it in d['objects']])

{'attack-pattern',
 'campaign',
 'course-of-action',
 'identity',
 'intrusion-set',
 'malware',
 'marking-definition',
 'relationship',
 'tool',
 'x-mitre-collection',
 'x-mitre-data-component',
 'x-mitre-data-source',
 'x-mitre-matrix',
 'x-mitre-tactic'}

### relationship

In [44]:
# описание конкретных атак со ссылками на сообщения - 165 штук
rel_l = [it for it in d['objects'] if it['type']=='relationship']
len(rel_l)

19438

In [47]:
set([it['relationship_type'] for it in rel_l])

{'attributed-to',
 'detects',
 'mitigates',
 'revoked-by',
 'subtechnique-of',
 'uses'}

#### subtechnique-of

In [49]:
subtech_l = [it for it in rel_l if it['relationship_type']=='subtechnique-of']

In [52]:
subtech_l[0]

{'object_marking_refs': ['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'],
 'id': 'relationship--00814703-3c3b-4872-89e9-cea4ba5edf2d',
 'type': 'relationship',
 'created': '2020-02-11T20:36:06.423Z',
 'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'modified': '2022-04-25T14:00:00.188Z',
 'relationship_type': 'subtechnique-of',
 'source_ref': 'attack-pattern--8d7bd4f5-3a89-4453-9c82-2c8894d5655e',
 'target_ref': 'attack-pattern--435dfb86-2697-4867-85b5-2fef496c0517',
 'x_mitre_version': '1.0',
 'x_mitre_modified_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'spec_version': '2.1',
 'x_mitre_attack_spec_version': '2.1.0',
 'x_mitre_domains': ['enterprise-attack']}

In [53]:
[it for it in subtech_l if it['source_ref']=='attack-pattern--4ff5d6a8-c062-4c68-a778-36fc5edd564f']

[{'object_marking_refs': ['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'],
  'id': 'relationship--5b0a440d-575a-4c1a-97aa-56945686f85a',
  'type': 'relationship',
  'created': '2020-03-27T21:06:33.285Z',
  'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
  'modified': '2022-04-25T14:00:00.188Z',
  'relationship_type': 'subtechnique-of',
  'source_ref': 'attack-pattern--4ff5d6a8-c062-4c68-a778-36fc5edd564f',
  'target_ref': 'attack-pattern--457c7820-d331-465a-915e-42f85500ccc4',
  'x_mitre_version': '1.0',
  'x_mitre_modified_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
  'spec_version': '2.1',
  'x_mitre_attack_spec_version': '2.1.0',
  'x_mitre_domains': ['enterprise-attack']}]

In [54]:
[it for it in attack_l if it['id']=='attack-pattern--457c7820-d331-465a-915e-42f85500ccc4']

[{'modified': '2024-03-01T16:25:43.150Z',
  'name': 'System Binary Proxy Execution',
  'description': 'Adversaries may bypass process and/or signature-based defenses by proxying execution of malicious content with signed, or otherwise trusted, binaries. Binaries used in this technique are often Microsoft-signed files, indicating that they have been either downloaded from Microsoft or are already native in the operating system.(Citation: LOLBAS Project) Binaries signed with trusted digital certificates can typically execute on Windows systems protected by digital signature validation. Several Microsoft signed binaries that are default on Windows installations can be used to proxy execution of other files or commands.\n\nSimilarly, on Linux systems adversaries may abuse trusted binaries such as <code>split</code> to proxy execution of malicious commands.(Citation: split man page)(Citation: GTFO split)',
  'kill_chain_phases': [{'kill_chain_name': 'mitre-attack',
    'phase_name': 'defens

### marking-definition

In [42]:
# описание конкретных атак со ссылками на сообщения - 165 штук
mark_l = [it for it in d['objects'] if it['type']=='marking-definition']
len(mark_l)

1

In [43]:
mark_l[0]

{'definition': {'statement': 'Copyright 2015-2024, The MITRE Corporation. MITRE ATT&CK and ATT&CK are registered trademarks of The MITRE Corporation.'},
 'id': 'marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168',
 'type': 'marking-definition',
 'created': '2017-06-01T00:00:00.000Z',
 'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'definition_type': 'statement',
 'x_mitre_attack_spec_version': '2.1.0',
 'spec_version': '2.1',
 'x_mitre_domains': ['enterprise-attack']}

### malware

In [40]:
# описание конкретных атак со ссылками на сообщения - 165 штук
mal_l = [it for it in d['objects'] if it['type']=='malware']
len(mal_l)

596

In [41]:
mal_l[0]

{'modified': '2023-04-04T20:20:59.961Z',
 'name': 'HDoor',
 'description': '[HDoor](https://attack.mitre.org/software/S0061) is malware that has been customized and used by the [Naikon](https://attack.mitre.org/groups/G0019) group. (Citation: Baumgartner Naikon 2015)',
 'x_mitre_platforms': ['Windows'],
 'x_mitre_deprecated': False,
 'x_mitre_domains': ['enterprise-attack'],
 'x_mitre_version': '1.0',
 'x_mitre_aliases': ['HDoor', 'Custom HDoor'],
 'type': 'malware',
 'id': 'malware--007b44b6-e4c5-480b-b5b9-56f2081b1b7b',
 'created': '2017-05-31T21:32:40.801Z',
 'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'revoked': False,
 'external_references': [{'source_name': 'mitre-attack',
   'url': 'https://attack.mitre.org/software/S0061',
   'external_id': 'S0061'},
  {'source_name': 'Baumgartner Naikon 2015',
   'description': 'Baumgartner, K., Golovkin, M.. (2015, May). The MsnMM Campaigns: The Earliest Naikon APT Campaigns. Retrieved April 10, 2019.',
   'url': 'htt

### intrusion-set

In [38]:
# описание конкретных атак со ссылками на сообщения - 165 штук
intr_l = [it for it in d['objects'] if it['type']=='intrusion-set']
len(intr_l)

165

In [39]:
intr_l[0]

{'modified': '2024-04-17T22:08:29.146Z',
 'name': 'APT38',
 'description': '[APT38](https://attack.mitre.org/groups/G0082) is a North Korean state-sponsored threat group that specializes in financial cyber operations; it has been attributed to the Reconnaissance General Bureau.(Citation: CISA AA20-239A BeagleBoyz August 2020) Active since at least 2014, [APT38](https://attack.mitre.org/groups/G0082) has targeted banks, financial institutions, casinos, cryptocurrency exchanges, SWIFT system endpoints, and ATMs in at least 38 countries worldwide. Significant operations include the 2016 Bank of Bangladesh heist, during which [APT38](https://attack.mitre.org/groups/G0082) stole $81 million, as well as attacks against Bancomext (Citation: FireEye APT38 Oct 2018) and Banco de Chile (Citation: FireEye APT38 Oct 2018); some of their attacks have been destructive.(Citation: CISA AA20-239A BeagleBoyz August 2020)(Citation: FireEye APT38 Oct 2018)(Citation: DOJ North Korea Indictment Feb 2021)(Ci

### identity

In [37]:
ident_l = [it for it in d['objects'] if it['type']=='identity']
len(ident_l)

1

### campaign

In [36]:
# описание конкретных атак со ссылками на сообщения, но очень мало
camp_l = [it for it in d['objects'] if it['type']=='campaign']
len(camp_l)

28

### тактика

In [32]:
tactic_l = [it for it in d['objects'] if it['type']=='x-mitre-tactic']
tactic_l[0]

{'x_mitre_domains': ['enterprise-attack'],
 'object_marking_refs': ['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'],
 'id': 'x-mitre-tactic--2558fd61-8c75-4730-94c4-11926db2a263',
 'type': 'x-mitre-tactic',
 'created': '2018-10-17T00:14:20.652Z',
 'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
 'external_references': [{'external_id': 'TA0006',
   'url': 'https://attack.mitre.org/tactics/TA0006',
   'source_name': 'mitre-attack'}],
 'modified': '2022-04-25T14:00:00.188Z',
 'name': 'Credential Access',
 'description': 'The adversary is trying to steal account names and passwords.\n\nCredential Access consists of techniques for stealing credentials like account names and passwords. Techniques used to get credentials include keylogging or credential dumping. Using legitimate credentials can give adversaries access to systems, make them harder to detect, and provide the opportunity to create more accounts to help achieve their goals.',
 'x_mitre_version': '1

In [33]:
[it for it in tactic_l if 'Execution' in it['name']]

[{'x_mitre_domains': ['enterprise-attack'],
  'object_marking_refs': ['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'],
  'id': 'x-mitre-tactic--4ca45d45-df4d-4613-8980-bac22d278fa5',
  'type': 'x-mitre-tactic',
  'created': '2018-10-17T00:14:20.652Z',
  'created_by_ref': 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5',
  'external_references': [{'external_id': 'TA0002',
    'url': 'https://attack.mitre.org/tactics/TA0002',
    'source_name': 'mitre-attack'}],
  'modified': '2022-04-25T14:00:00.188Z',
  'name': 'Execution',
  'description': 'The adversary is trying to run malicious code.\n\nExecution consists of techniques that result in adversary-controlled code running on a local or remote system. Techniques that run malicious code are often paired with techniques from all other tactics to achieve broader goals, like exploring a network or stealing data. For example, an adversary might use a remote access tool to run a PowerShell script that does Remote System Discovery. 

### пример

In [ ]:
it = d['objects'][1000]
it['type'], it['name'], it['description'], it['x_mitre_attack_spec_version'], it['x_mitre_version']

In [ ]:
it['object_marking_refs']

### поиск атаки

In [ ]:
text = open('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/enterprise-attack.json').read()

In [ ]:
len(text)

In [ ]:
ch = text.find('Adversaries may execute active reconnaissance scans to gather information that can be used during targeting.')

In [ ]:
text[ch-1200:ch+100]

## возьмем атаки

In [ ]:
attack_l = [it for it in d['objects'] if it['type']=='attack-pattern']
len(attack_l)

In [ ]:
attack_l[0]


In [ ]:

# 1406, 1575
[it for it in attack_l if it['external_references'][0]['external_id']=='T1557.001']

In [ ]:
from itertools import chain
set(chain(*[it['x_mitre_domains'] for it in attack_l]))

In [ ]:
enterp_attack_l = [it for it in attack_l if 'enterprise-attack' in it['x_mitre_domains']]

enterp_attack_l это все, которые попали в наш фильтр, может тогда фильтр слишком жесткий?

In [ ]:
len(enterp_attack_l)

In [ ]:
# https://attack.mitre.org/techniques/T1059/
s = 'Adversaries may scan victim IP blocks to gather information that can be used during targeting.'
attack = [it for it in attack_l if s in it['description']]
attack

In [ ]:
attack[0]['description'], attack[0]['external_references'][0]['external_id'], attack[0]['external_references'][0]['url']

In [ ]:
mitre_df = pd.DataFrame([(attack['description'], attack['external_references'][0]['external_id'], attack['external_references'][0]['url']) for attack in attack_l],
                        columns=['sentence', 'labels', 'url'])

In [ ]:
s = 'Adversaries may scan victim IP blocks to gather information that can be used during targeting.'
mitre_df.query('sentence.str.contains(@s)')

In [ ]:
url = 'https://attack.mitre.org/tactics/TA0006/'
mitre_df.query('url==@url')

In [ ]:
mitre_df['labels']

## ищем по тексту

In [ ]:
text = open('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/enterprise-attack.json').read()

In [ ]:
len(text)

In [ ]:
s = 'has used watering hole attacks, often with zero-day exploits, to gain initial access'
ch = text.find(s)

In [ ]:
text[ch-150:ch+300]

In [ ]:
'https://attack.mitre.org/techniques/T1059/002/'

## ищем признак группы

In [ ]:
descr1 = 'may circumvent mechanisms designed to control elevate privileges to gain higher-level permissions.'
descr2 = 'Within Linux and MacOS systems, sudo'
descr3 = 'Adversaries may gain access and continuously communicate with victims by injecting malicious content into systems through online network traffic'

attack1 = [it for it in attack_l if descr1 in it['description']][0]
attack2 = [it for it in attack_l if descr2 in it['description']][0]
attack3 = [it for it in attack_l if descr3 in it['description']][0]

In [ ]:
attack2
# identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5

In [ ]:
kill_chain_phases


## Удалим ссылки из описания и соберем в отдельном поле

In [ ]:
mitre_df['descr_links'] = mitre_df['sentence'].str.findall('\((https://[^\s]{1,})\)')

In [ ]:
mitre_df['sentence'] = mitre_df['sentence'].str.replace(r'\(https://[^\s]{1,}\)', r'', regex=True)
mitre_df

### ссылки из процедур и митигаций

In [ ]:
attack_sample = mitre_df.query('labels=="T1548.003" and not proc_flag')
display(attack_sample)
id = attack_sample['id'].iloc[0]

In [ ]:
attack = [it for it in attack_l if it['id']==id][0]
attack

In [ ]:
attack['x_mitre_platforms'], attack['external_references'][1:], attack['kill_chain_phases'], attack['name'],
attack['x_mitre_data_sources'], attack['x_mitre_permissions_required'], attack['x_mitre_effective_permissions'], attack['created']

In [ ]:
[Proton](https://attack.mitre.org/software/S0279) modifies the tty_tickets line in the sudoers file.(Citation: objsee mac malware 2017)

In [ ]:
proc_software_links = []
proc_citation_links = []

proc_itms = [it for it in rel_l if (it['target_ref']==attack['id']) and (it['relationship_type']=='uses')]
it = proc_itms[1]
it

In [ ]:
import re
# proc_software_links, proc_descr_links
re.findall(r'\((https://[^\s\)]{1,})', it['description']), [subit['url'] for subit in it['external_references']]

In [ ]:
det_itms = [it for it in rel_l if (it['target_ref']==attack['id']) and (it['relationship_type']=='detects')]
it = det_itms[2]
# det_descr, det_data_comp
it['description'], it['source_ref']

In [ ]:
mit_itms = [it for it in rel_l if (it['target_ref']==attack['id']) and (it['relationship_type']=='mitigates')]
it = mit_itms[2]
it['description'], it['source_ref']

## ищем связанные процедуры с objectpath

In [ ]:
!pip install objectpath

In [ ]:
import objectpath as obp

query = obp.Tree(d)

### объект по описанию можно найти

In [ ]:
s = '[Andariel](https://attack.mitre.org/groups/G0138) has used watering hole attacks, often with zero-day exploits, to gain initial access to victims within a specific IP range.(Citation: AhnLab Andariel Subgroup of Lazarus June 2018)(Citation: TrendMicro New Andariel Tactics July 2018)'

res_descr = list(query.execute(f"$..*[@.description is '{s}']"))

In [ ]:
type(res_descr)

Связи с атакой:
- created_by_ref
- target_ref
- object_marking_refs

In [13]:
attack = [it for it in attack_l if 'Typical drive-by compromise process:' in it['description']][0]


#### created_by_ref
много очень

In [11]:
ident = 'identity--c78cb6e5-0c4b-4611-8297-d1b8b55e40b5'

res2 = list(query.execute(f"$..*[@.created_by_ref is '{ident}']"))


NameError: name 'query' is not defined

In [ ]:
res2[0].keys()

#### object_marking_refs
тут тоже дичь

In [19]:
pd.DataFrame([it['object_marking_refs'] for it in attack_l], columns=['val']).explode('val')['val'].unique()

array(['marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'],
      dtype=object)

Для всех техник атак один тип

In [12]:
object_marking_refs = 'marking-definition--fa42a846-8d90-4e51-bc29-71d5b4802168'

res2 = list(query.execute(f"$..*['{object_marking_refs}' in @.object_marking_refs]"))
res2

NameError: name 'query' is not defined

In [ ]:
len(res2[0])

In [ ]:
res2[0]['object_marking_refs']

### по id атаки

In [ ]:
res = list(query.execute("$..*[@.target_ref is 'attack-pattern--d742a578-d70e-4d0e-96a6-02a9c30204e6']"))

In [ ]:
{it['relationship_type'] for it in res}

Тут как раз все виды
- uses - процедуры
- detects - Detection
- mitigates - Mitigations

In [ ]:
(len([it for it in res if it['relationship_type']=='uses']), len([it for it in res if it['relationship_type']=='detects']),
len([it for it in res if it['relationship_type']=='mitigates']))

In [ ]:
[it for it in res if it['relationship_type']=='uses'][-1]

- некоторые примеры процедур почему-то на сайте не отображаются
  - [пример](https://attack.mitre.org/techniques/T1189/), а в ссылках есть (it['relationship_type']=='uses' с id атаки) и [тут](https://service.securitm.ru/mitre/techniques/T1189)

### поиск процедур
сначала нашел родителя, как в файле objectpath и обнаружил, что у него столько же элементов, сколько и у верхнеуровневого objects ключа и сделал вывод, что один

In [ ]:
obj_target_refs = [it for it in d['objects'] if 'target_ref' in it.keys()]

In [ ]:
pattern = 'attack-pattern--d742a578-d70e-4d0e-96a6-02a9c30204e6'


In [ ]:
rel_l = [it for it in d['objects'] if it['type']=='relationship']

По ходу это объекты с типом relationship

In [ ]:
len(obj_target_refs), len(rel_l)

количество совпало

In [ ]:
[it for it in rel_l if not 'target_ref' in it.keys()]

нет item-ов без ссылок атаку target_ref

In [ ]:
attack

## атаки одноименки

In [ ]:
import json
import re

d=json.load(open('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/enterprise-attack.json'))
# attack_l = [it for it in d['objects'] if it['type']=='attack-pattern']
attack_l = [it for it in d['objects'] if it['type']=='attack-pattern' if not 'revoked' in it.keys() or not it['revoked']]
rel_l = [it for it in d['objects'] if it['type']=='relationship']

mitre_df = pd.DataFrame([(attack['id'], attack['name'], attack['description'], attack['external_references'][0]['external_id'],
                          attack['external_references'][0]['url'], attack['x_mitre_platforms'], attack['kill_chain_phases'],
                          attack['created'], attack['external_references'][1:],
                          attack['x_mitre_permissions_required'] if 'x_mitre_permissions_required' in attack.keys() else None,
                         attack['x_mitre_effective_permissions'] if 'x_mitre_effective_permissions' in attack.keys() else None,
                          attack['x_mitre_data_sources'] if 'x_mitre_data_sources' in attack.keys() else None,
                          attack['x_mitre_defense_bypassed'] if 'x_mitre_defense_bypassed' in attack.keys() else None
                          )
                          for attack in attack_l],
                        columns=['id', 'name', 'sentence', 'labels', 'url', 'platforms', 'kill_chain_phases',
                                 'created', 'doc_refs', 'permissions', 'effective_permissions', 'data_sources', 'defense_bypassed'])
mitre_df['proc_flag'] = False


In [ ]:
mitre_df[mitre_df['name'].duplicated(keep=False)].sort_values(by='name').head()

## выделяем субтехники

In [ ]:
import json
import re

d=json.load(open('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/enterprise-attack.json'))
# attack_l = [it for it in d['objects'] if it['type']=='attack-pattern']
attack_l = [it for it in d['objects'] if it['type']=='attack-pattern' if not 'revoked' in it.keys() or not it['revoked']]

In [ ]:
len([it for it in attack_l if  it['x_mitre_is_subtechnique']==False])

## атаки с устаревшими ссылками

In [ ]:
d=json.load(open('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/enterprise-attack.json'))
attack_l = [it for it in d['objects'] if it['type']=='attack-pattern']

### T1067

In [ ]:
prb_l = [it for it in attack_l if it['name']=='Bootkit']
len(prb_l)

In [ ]:
prb_l[0]

In [ ]:
prb_l[1]

### T1066

In [ ]:
prb_l = [it for it in attack_l if it['name']=='Indicator Removal from Tools']
prb_l[0]

In [ ]:
prb_l[1]

- отличие флаг 'revoked': False (актуальная)

In [ ]:
mitre_df.query('labels.isin(["T1027.005", "T1066"])')

### проблемные id атак

In [ ]:
d=json.load(open('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/enterprise-attack.json'))
attack_l = [it for it in d['objects'] if it['type']=='attack-pattern']
mitre_df = pd.DataFrame([(attack['id'], attack['description'], attack['external_references'][0]['external_id'], attack['external_references'][0]['url']) for attack in attack_l],
                        columns=['id', 'sentence', 'labels', 'url'])

In [ ]:
pattern = 'attack-pattern--00d0b012-8a03-410e-95de-5826bf542de6'

obj_target_refs = [it for it in d['objects'] if 'target_ref' in it.keys()]

In [ ]:
mitre_df.query(f'id=="{pattern}"')

In [ ]:
attack = [it for it in attack_l if it['id']==pattern][0]
attack

In [ ]:
res = list(query.execute(f"$..*[@.target_ref is '{pattern}']"))

In [ ]:
{it['relationship_type'] for it in res}

## Итог

In [4]:
import json
import re
from ruamel.yaml import YAML
import pandas as pd


conf = YAML().load(open('params.yaml'))

In [62]:
# d=json.load(open('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/enterprise-attack.json'))

d=json.load(open(conf['get_data']['mitre_attack_fn']))

# attack_l = [it for it in d['objects'] if it['type']=='attack-pattern']
attack_l = [it for it in d['objects'] if it['type']=='attack-pattern' if not 'revoked' in it.keys() or not it['revoked']]
rel_l = [it for it in d['objects'] if it['type']=='relationship']
subtech_l = [it for it in d['objects'] if it['type']=='relationship' and it['relationship_type']=='subtechnique-of']

mitre_df = pd.DataFrame([(attack['id'], attack['name'], attack['description'], attack['external_references'][0]['external_id'],
                          attack['external_references'][0]['url'], attack['x_mitre_is_subtechnique'], attack['x_mitre_platforms'], attack['kill_chain_phases'],
                          attack['created'], attack['external_references'][1:],
                          attack['x_mitre_permissions_required'] if 'x_mitre_permissions_required' in attack.keys() else None,
                         attack['x_mitre_effective_permissions'] if 'x_mitre_effective_permissions' in attack.keys() else None,
                          attack['x_mitre_data_sources'] if 'x_mitre_data_sources' in attack.keys() else None,
                          attack['x_mitre_defense_bypassed'] if 'x_mitre_defense_bypassed' in attack.keys() else None
                          )
                          for attack in attack_l],
                        columns=['id', 'name', 'sentence', 'labels', 'url', 'subtechnique', 'platforms', 'kill_chain_phases',
                                 'created', 'doc_refs', 'permissions', 'effective_permissions', 'data_sources', 'defense_bypassed'])
mitre_df['proc_flag'] = False


In [66]:
from itertools import chain
mittre_l = []
for _, row in mitre_df.iterrows():

    proc_itms = [it for it in rel_l if (it['target_ref']==row['id']) and (it['relationship_type']=='uses')]
    det_itms = [it for it in rel_l if (it['target_ref']==row['id']) and (it['relationship_type']=='detects')]
    mit_itms = [it for it in rel_l if (it['target_ref']==row['id']) and (it['relationship_type']=='mitigates')]


    part_df = row.drop(['proc_flag']).to_frame().T
    part_df['proc_software_links'] = [list(chain(*[re.findall(r'\((https://[^\s\)]{1,})', it['description']) for it in proc_itms ]))]

    
    if part_df['subtechnique'].iloc[0]:
        part_df['par_id'] = [it for it in subtech_l if it['source_ref']==row['id']][0]['target_ref']
    else:
        part_df['par_id'] = None
    part_df['proc_descr_links'] = [[subit['url'] for it in proc_itms if 'external_references' in it for subit in it['external_references'] ]]
    part_df['proc_descr'] = [[it['description'] for it in proc_itms]]
    # break

    part_df['det_descr'] = [[it['description'] for it in det_itms if 'description' in it]]
    part_df['det_data_comp'] = [[it['source_ref'] for it in det_itms if 'source_ref' in it]]
    part_df['mit_descr'] = [[it['description'] for it in mit_itms if 'description' in it]]
    part_df['mit_coa'] = [[it['source_ref'] for it in mit_itms if 'source_ref' in it]]

    mittre_l.append(part_df)




In [67]:
mitre_df = pd.concat(mittre_l, ignore_index=True)


In [68]:
mitre_df['kill_chain_tags'] = mitre_df['kill_chain_phases'].map(lambda x: [it['phase_name'] for it in x])

In [70]:
mitre_df.shape

(649, 23)

In [75]:
mitre_df = mitre_df.merge(mitre_df[['id', 'name']].rename(columns={'id':'par_id', 'name':'par_name'}), on='par_id', how='left')
mitre_df.head(2)

,id,name,sentence,labels,url,subtechnique,platforms,kill_chain_phases,created,doc_refs,...,proc_software_links,par_id,proc_descr_links,proc_descr,det_descr,det_data_comp,mit_descr,mit_coa,kill_chain_tags,par_name
0,attack-pattern--0042a9f5-f053-4769-b3ef-9ad018...,Extra Window Memory Injection,Adversaries may inject malicious code into pro...,T1055.011,https://attack.mitre.org/techniques/T1055/011,True,[Windows],"[{'kill_chain_name': 'mitre-attack', 'phase_na...",2020-01-14T17:18:32.126Z,[{'url': 'https://msdn.microsoft.com/library/w...,...,"[https://attack.mitre.org/software/S0091, http...",attack-pattern--43e7dc91-05b2-474c-b9ac-2ed4fe...,[https://recon.cx/2018/brussels/resources/slid...,[[Epic](https://attack.mitre.org/software/S009...,[Monitor for API calls related to enumerating ...,[x-mitre-data-component--9bde2f9d-a695-4344-bf...,[Some endpoint security solutions can be confi...,[course-of-action--90f39ee1-d5a3-4aaa-9f28-3b4...,"[defense-evasion, privilege-escalation]",Process Injection
1,attack-pattern--005a06c6-14bf-4118-afa0-ebcd8a...,Scheduled Task,Adversaries may abuse the Windows Task Schedul...,T1053.005,https://attack.mitre.org/techniques/T1053/005,True,[Windows],"[{'kill_chain_name': 'mitre-attack', 'phase_na...",2019-11-27T14:58:00.429Z,"[{'source_name': 'ProofPoint Serpent', 'descri...",...,"[https://attack.mitre.org/software/S0588, http...",attack-pattern--35dd844a-b219-4e2b-a6bb-efa9a7...,[https://www.microsoft.com/security/blog/2021/...,[[GoldMax](https://attack.mitre.org/software/S...,[Monitor for newly constructed registry keys u...,[x-mitre-data-component--7f70fae7-a68d-4730-a8...,[Configure the Increase Scheduling Priority op...,[course-of-action--9bb9e696-bff8-4ae1-9454-961...,"[execution, persistence, privilege-escalation]",Scheduled Task/Job


In [20]:
mitre_df.to_csv(conf['get_data']['data_mitre_fn'], index=False)
# import joblib
# joblib.dump(mitre_df, '/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/mitre_df.pkl')

### развертывание mitr

In [3]:
import pandas as pd

In [23]:
mitre_df = pd.read_csv(conf['get_data']['data_mitre_fn'])

type_list_cols = ['platforms', 'kill_chain_phases', 'doc_refs', 'permissions', 'effective_permissions', 'data_sources', 'defense_bypassed','proc_software_links', 'proc_descr_links', 'proc_descr', 'det_descr',
'det_data_comp', 'mit_descr', 'mit_coa', 'kill_chain_tags']

for col in type_list_cols:
    mitre_df[col] = mitre_df[col].map(lambda x: eval(x), na_action='ignore')

# import joblib
# mitre_df = joblib.load('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/mitre_df.pkl')


In [28]:
proc_df = mitre_df[['id', 'name', 'url', 'labels', 'proc_descr', 'kill_chain_tags', 'subtechnique']].explode('proc_descr')\
                  .rename(columns={'proc_descr':'sentence'})\
                  .assign(is_proc=True)
main_descr_df = mitre_df[['id', 'name', 'url', 'labels', 'sentence', 'kill_chain_tags', 'subtechnique']].assign(is_proc=False)
mitre_attack_df = pd.concat([main_descr_df, proc_df], ignore_index=True)

In [29]:
# удаляем строки, где процедуры не было совсем
mitre_attack_df = mitre_attack_df[~mitre_attack_df['sentence'].isna()].reset_index(drop=True)


# удалим ссылочки
mitre_attack_df['sentence'] = mitre_attack_df['sentence'].str.replace(r'\(https://[^\s]{1,}\)', r'', regex=True)
mitre_attack_df.head(2)

,id,name,url,labels,sentence,kill_chain_tags,subtechnique,is_proc
0,attack-pattern--0042a9f5-f053-4769-b3ef-9ad018...,Extra Window Memory Injection,https://attack.mitre.org/techniques/T1055/011,T1055.011,Adversaries may inject malicious code into pro...,"[defense-evasion, privilege-escalation]",True,False
1,attack-pattern--005a06c6-14bf-4118-afa0-ebcd8a...,Scheduled Task,https://attack.mitre.org/techniques/T1053/005,T1053.005,Adversaries may abuse the Windows Task Schedul...,"[execution, persistence, privilege-escalation]",True,False


In [30]:
label2tactic = mitre_attack_df.set_index('labels')['kill_chain_tags'].to_dict()

#### dumps

In [33]:
import json

with open(conf['get_data']['label2tactic_fn'], 'wt') as f_wr:
    json.dump(label2tactic, f_wr)
# joblib.dump(label2tactic, '/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/label2tactic.pkl')

In [ ]:
mitre_attack_df[['sentence', 'labels', 'url']].to_csv(conf['get_data']['data_mitre_attack_proc_fn'], index=False)

# joblib.dump(mitre_attack_df[['sentence', 'labels']]\
#         .assign(labels = mitre_attack_df['labels'].map(lambda x:[x])), '/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/mitre_source.pkl')

In [40]:
mitre_attack_df = pd.read_csv(conf['get_data']['data_mitre_attack_proc_fn'])\
                    .assign(labels = mitre_attack_df['labels'].map(lambda x:[x]))
mitre_attack_df

,sentence,labels,url
0,Adversaries may inject malicious code into pro...,[T1055.011],https://attack.mitre.org/techniques/T1055/011
1,Adversaries may abuse the Windows Task Schedul...,[T1053.005],https://attack.mitre.org/techniques/T1053/005
2,Adversaries may attach filters to a network so...,[T1205.002],https://attack.mitre.org/techniques/T1205/002
3,Adversaries may use utilities to compress and/...,[T1560.001],https://attack.mitre.org/techniques/T1560/001
4,Adversaries may use [Valid Accounts] to remote...,[T1021.005],https://attack.mitre.org/techniques/T1021/005
...,...,...,...
14353,[DustySky] can shutdown the infected machine.(...,[T1529],https://attack.mitre.org/techniques/T1529
14354,[Cobalt Strike] can use spoof arguments in spa...,[T1564.010],https://attack.mitre.org/techniques/T1564/010
14355,[SombRAT] has the ability to modify its proces...,[T1564.010],https://attack.mitre.org/techniques/T1564/010
14356,[Blue Mockingbird] has used wmic.exe and Windo...,[T1574.012],https://attack.mitre.org/techniques/T1574/012


In [34]:
with open(conf['get_data']['label2tactic_fn'], 'rt') as f_r:
    label2tactic = json.load(f_r)

### другие файлы


In [21]:
import pandas as pd
import joblib
import numpy as np

False

#### tram


In [30]:
# tram_df = pd.read_json('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/scibert/multi_label.json').drop(columns='doc_title')


tram_df = pd.read_json(conf['get_data']['tram_fn']).drop(columns='doc_title')
tram_df

,sentence,labels
0,title: NotPetya Technical Analysis – A Triple ...,[]
1,Executive Summary This technical analysis prov...,[]
2,For more information on CrowdStrike’s proactiv...,[]
3,NotPetya combines ransomware with the ability ...,[]
4,It spreads to Microsoft Windows machines using...,[T1210]
...,...,...
19173,[2] Eclypsium Blog - TrickBot Now Offers 'Tric...,[]
19174,"Initial Version March 24, 2021:",[]
19175,Added MITRE ATT&CK Technique T1592.003 used fo...,[]
19176,Added new MITRE ATT&CKs and updated Table 1,[]


#### sec_bert


In [28]:
# mitr_alt_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/dataset.txt')
mitr_alt_data = pd.read_csv(conf['get_data']['mitre_alt_fn'])

mitr_alt_data['labels'] = mitr_alt_data[['label_subtec', 'label_tec']].apply(lambda x: list(set((x[0], x[1]))), axis=1)
mitr_alt_data = mitr_alt_data[['sentence', 'labels']]

C:\Users\admin\AppData\Local\Temp\ipykernel_16512\1707112105.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  mitr_alt_data['labels'] = mitr_alt_data[['label_subtec', 'label_tec']].apply(lambda x: list(set((x[0], x[1]))), axis=1)


In [ ]:
joblib.dump(mitr_alt_data[['sentence', 'labels']], '/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/mitre_alt.pkl')

In [31]:
if conf['get_data']['use_alt_mitre']:
  mitr_df = mitr_alt_data
else:
  mitr_df = mitre_attack_df

In [ ]:
rep_data  = joblib.load('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/rep_data.pkl')
rep_data.head(2)

In [ ]:
df = pd.concat([mitr_df, tram_df], ignore_index=True)

In [ ]:
https://attack.mitre.org/techniques/T1574/002/

In [ ]:
with pd.option_context('display.max_colwidth', 500):
  display(tram_df.head())

In [ ]:
df = pd.concat([df, rep_data[['sentence', 'labels']]], ignore_index=True)

In [ ]:
target = 'tactic'
if target == 'tactic':
  from itertools import chain
  label2tactic = joblib.load('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/label2tactic.pkl')

  df['labels'] = df['labels'].map(lambda x: list(chain(*[label2tactic[it] for it in x])))


joblib.dump(df, '/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/mitre_full_df.pkl')

### небольшая eda

In [ ]:
mitre_df.head(2)

#### data_sources

In [ ]:
mitre_df['data_sources'].isna().sum(), mitre_df['data_sources'].isna().mean()

In [ ]:
temp_df = mitre_df.explode('data_sources')
temp_df['data_sources'].nunique() , temp_df.shape[0], temp_df['data_sources'].nunique()/temp_df.shape[0]

In [ ]:
temp_df['data_sources'].duplicated().mean()

- Мало пропусков, 100 уникальных и почти каждая дублируется, может хорошо разделить данные

#### permissions

In [ ]:
mitre_df['effective_permissions'].isna().mean(), mitre_df['permissions'].isna().mean()

In [ ]:
temp_df = mitre_df.explode('permissions')
temp_df['permissions'].nunique() , temp_df.shape[0], temp_df['permissions'].nunique()/temp_df.shape[0]

- Мало записей с разрешениями
- уникальных мало, поэтому можно добавить к эмбеддингу

#### defense_bypassed

In [ ]:
mitre_df['defense_bypassed'].isna().sum(), mitre_df['defense_bypassed'].isna().mean()

In [ ]:
temp_df = mitre_df.explode('defense_bypassed')
temp_df['defense_bypassed'].nunique() , temp_df.shape[0], temp_df['defense_bypassed'].nunique()/temp_df.shape[0]

- Мало записей с обходом защиты
- уникальных 39, поэтому можно добавить к эмбеддингу

#### mit coa

In [ ]:
#	mit_coa
mitre_df.explode('mit_coa')

In [ ]:
mitre_df.explode('mit_coa')['mit_coa'].value_counts().loc[lambda x: x==1].sum(), mitre_df.explode('mit_coa').shape[0]

#### det_data_comp

In [ ]:
mitre_df.explode('det_data_comp')['det_data_comp'].value_counts().loc[lambda x: x==1].sum(), mitre_df.explode('det_data_comp').shape[0]

#### proc_descr_links

In [ ]:
mitre_df.explode('proc_descr_links')['proc_descr_links'].value_counts().loc[lambda x: x==1].sum(), mitre_df.explode('proc_descr_links').shape[0]

proc_descr_links содержит дубли если шо

#### proc_software_links

In [ ]:
# столько ссылок software встречаются однократно
mitre_df.explode('proc_software_links')['proc_software_links'].value_counts().loc[lambda x: x==1].sum()/mitre_df.explode('proc_software_links').shape[0]

In [ ]:
mitre_df.explode('proc_software_links')['proc_software_links'].nunique(), mitre_df.explode('proc_software_links').shape[0]

#### doc_refs

In [ ]:
# столько ссылок встречаются однократно
mitre_df.explode('doc_refs')['doc_refs'].value_counts().loc[lambda x: x==1].sum()/mitre_df.explode('doc_refs').shape[0], mitre_df.explode('doc_refs').shape[0]

In [ ]:
mitre_df['platforms'].explode().unique()

In [ ]:
mitre_df['kill_chain_tags'].explode().unique()

In [ ]:
# где пусто, там пустые списки
for col in ['proc_software_links', 'det_data_comp', 'mit_coa', 'proc_descr_links', 'doc_refs', 'platforms', 'kill_chain_tags']:

  display(mitre_df[mitre_df[col].map(lambda x: (len(x)==0) & (not isinstance(x, list)))])

- doc_refs немного, но повторения встречаютс только для половины
- proc_software_links в 10 раз больше, повторяются почти все
- proc_descr_links около 18 тыс. записей,
- det_data_comp, mit_coa поменьше можно

включаем в общий датасет пока proc_software_links, det_data_comp, mit_coa  и сцепляем. platforms и kill_chain_tags можно потом добавить one hot

#### subtechnique

In [ ]:
mitre_attack_df.loc[(mitre_attack_df.subtechnique!=True), 'labels'].str.len().describe()

In [ ]:
mitre_attack_df.loc[(mitre_attack_df.subtechnique==True), 'labels'].str.len().describe()

### после вытягивания процедур

In [ ]:
mitre_df['descr_links'] = mitre_df['sentence'].str.findall('\((https://[^\s]{1,})\)')

In [ ]:
mitre_df['sentence'] = mitre_df['sentence'].str.replace(r'\(https://[^\s]{1,}\)', r'', regex=True)
mitre_df

In [ ]:
with pd.option_context('display.max_colwidth', 400):
  display(mitre_df.query('sentence.str.contains("Binary padding effectively changes")'))

In [ ]:
with pd.option_context('display.max_colwidth', 400):
  display(mitre_df.query('id=="attack-pattern--5bfccc3f-2326-4112-86cc-c1ece9d8a2b5"'))

у кого нет процедур, возможно, ссылаются на другие атаки, у кого есть

## Проблемы

### одинаковое описание разные атаки

- https://attack.mitre.org/techniques/T1009
- https://attack.mitre.org/techniques/T1027.001

удалим почти дубли, надо подумать как

### на сайте чего-то нет, но есть в данных (правильно?)

- некоторые примеры процедур почему-то на сайте не отображаются
  - [пример](https://attack.mitre.org/techniques/T1189/), а в ссылках есть (it['relationship_type']=='uses' с id атаки) и [тут](https://service.securitm.ru/mitre/techniques/T1189)

### нет процедур

- хотя ссылаются на другие атаки
  - https://attack.mitre.org/techniques/T1066
  T1067

In [ ]:
mitre_df.query('labels.isin(["T1027.005", "T1066"])')[['name', 'id', 'url', 'mit_coa', 'kill_chain_tags']]

Одна запись ссылается на другую при этом у них разные mit_coa

- как оценить похожесть текстов

## найдем похожие атаки

- по разделу
- по общим сссылкам
- по общим описаниям

# tram

In [2]:
import pandas as pd

In [24]:
tram_data = pd.read_json(conf['get_data']['tram_fn'])
tram_data

,sentence,labels,doc_title
0,title: NotPetya Technical Analysis – A Triple ...,[],NotPetya Technical Analysis – A Triple Threat:...
1,Executive Summary This technical analysis prov...,[],NotPetya Technical Analysis – A Triple Threat:...
2,For more information on CrowdStrike’s proactiv...,[],NotPetya Technical Analysis – A Triple Threat:...
3,NotPetya combines ransomware with the ability ...,[],NotPetya Technical Analysis – A Triple Threat:...
4,It spreads to Microsoft Windows machines using...,[T1210],NotPetya Technical Analysis – A Triple Threat:...
...,...,...,...
19173,[2] Eclypsium Blog - TrickBot Now Offers 'Tric...,[],AA21-076A: TrickBot Malware
19174,"Initial Version March 24, 2021:",[],AA21-076A: TrickBot Malware
19175,Added MITRE ATT&CK Technique T1592.003 used fo...,[],AA21-076A: TrickBot Malware
19176,Added new MITRE ATT&CKs and updated Table 1,[],AA21-076A: TrickBot Malware


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/data_df.csv')
data['labels'] = data['labels'].map(lambda x: eval(x))
data

In [ ]:
data[data['labels'].map(lambda x: 'T1027.005' in x)]

In [ ]:
mitr_data[mitr_data['labels'].map(lambda x: 'T1027.005' in x)]

# sec bert


In [ ]:
mitr_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/dataset.txt')
mitr_data['labels'] = mitr_data[['label_subtec', 'label_tec']].apply(lambda x: list(set((x[0], x[1]))), axis=1)
mitr_data = mitr_data[['sentence', 'labels']]

In [ ]:
joblib.dump(mitr_data[['sentence', 'labels']], '/content/drive/MyDrive/Colab Notebooks/texts/sec_bert/data/mitre_alt.pkl')